In [1]:
data_var = '2024-01-05'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10499,2024-01-05,Noruega Blno,15:30,Ammerud,Nidaros Jets,1.67,2.12,164.5,1.86,1.80,-3.5,2.00,1.72,tdBrOYQ5,0.598802,0.471698,0.537634,0.555556,0.070501,429.546,301.541260,0.702000,0.6,1.341641,2.236068,115.20,5.622,3.748402,0.666738,-82.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,70,1.44,2.06,0.000,0.000,0.167915,0.023184,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
10500,2024-01-05,Polônia Liga De Basquete,15:30,Ostrow Wielkopolski,Dziki Warszawa,1.13,5.50,161.5,1.86,1.88,-13.5,2.05,1.68,pOnocrwd,0.884956,0.181818,0.537634,0.531915,0.066774,168.962,38.050680,0.225203,2.4,1.341641,0.559017,119.60,1.828,0.553597,0.302843,77.0,157.422,30.975915,0.196770,1.8,1.643168,0.912871,153.44,1.992,0.498468,0.250235,2.0,92,56,1.30,2.74,124.056,262.738,0.932144,0.007563,0.140284,-0.49,-0.098,-1.326531,0.0,0.0,0.0,-2.99,-0.598,-7.525084,0.000000,0.0,0.000000
10501,2024-01-05,Polônia Liga De Basquete,16:30,Trefl Sopot,Anwil Wloclawek,1.94,1.81,162.5,1.86,1.80,-1.5,2.09,1.66,YqTuJPaF,0.515464,0.552486,0.537634,0.555556,0.067950,139.390,47.261374,0.339059,2.4,1.341641,0.559017,213.36,1.556,0.591718,0.380281,44.0,116.178,22.163382,0.190771,2.4,1.341641,0.559017,123.67,1.354,0.180361,0.133206,54.0,84,83,2.54,1.49,118.178,0.000,0.049026,0.023184,0.162163,-0.76,-0.152,-6.184211,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
10502,2024-01-05,Portugal Lpb,16:00,FC Porto,CD Póvoa,1.01,12.20,157.5,1.87,1.79,-17.5,2.00,1.72,pGKI9Sgm,0.990099,0.081967,0.534759,0.558659,0.072066,109.726,44.699485,0.407374,2.4,1.341641,0.559017,188.81,1.322,0.597888,0.452260,55.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,58,2.39,8.65,0.000,0.000,1.197960,0.030912,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
10503,2024-01-05,Reino Unido Troféu Da Bbl,16:00,Leicester Riders,Bristol Flyers,1.41,2.76,167.5,1.85,1.85,-6.5,2.01,1.72,4CXyW2ns,0.709220,0.362319,0.540541,0.540541,0.071539,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.457839,0.000000,0.109952,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,2024-01-05,Dinamarca Liga De Basquete,15:00,Bears Academy,Værløse,2.49,1.51,159.5,1.80,1.87,2.5,2.10,1.65,vcGBl19G,0.401606,0.662252,0.555556,0.534759,0.063858,336.642,318.528575,0.946194,0.6,1.341641,2.236068,854.70,4.902,4.317635,0.880790,-52.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,77,84,11.10,2.99,0.000,0.000,0.346482,0.026974,0.169706,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.000000,0.0,0.000000
10613,2024-01-05,Croácia Premijer Liga,14:00,Alkar,GKK Sibenka,2.10,1.68,156.5,1.85,1.85,1.5,1.94,1.77,pImb9J34,0.476190,0.595238,0.540541,0.540541,0.071429,390.332,244.183456,0.625579,0.6,1.341641,2.236068,689.50,5.590,3.281821,0.587088,-52.0,360.270,287.546065,0.798140,1.2,1.643168,1.369306,78

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
10499,15:30,Noruega Blno,Ammerud,Nidaros Jets,1.67,Back Home
10501,16:30,Polônia Liga De Basquete,Trefl Sopot,Anwil Wloclawek,1.94,Back Home
10505,16:30,Reino Unido Troféu Da Bbl,Caledonia Gladiators,Cheshire Phoenix,1.71,Back Home
10506,15:00,República Tcheca 1. Liga,Pardubice 2,Opava 2,2.20,Back Home
10514,07:05,Japão B.League,Hiroshima D.,Akita,1.60,Back Home
10515,07:30,Israel Liga Leumit,Gilboa Galil,Ironi Nahariya,1.75,Back Home
10518,14:00,Hungria Nacional,KTE-Duna,DEAC,1.67,Back Home
10521,12:30,Finlândia Divisão I Feminina,Puhuttaret F,Lappeenranta F,2.06,Back Home
10523,15:00,República Tcheca 1. Liga,Prostejov,Snakes Ostrava,2.01,Back Home
10531,21:00,Eua Ncaa,St. Peters,Iona,2.14,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO UNDER V1 (QCUT) ------------



,Time,League,Home,Away,Odds_Under,Bet
10565,16:30,Europa Euroliga,Valencia,Panathinaikos,1.95,Back Home


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
10518,14:00,Hungria Nacional,KTE-Duna,DEAC,1.67,Back Home
10533,21:00,Eua Ncaa,Mt. St. Marys,Canisius,1.69,Back Home
10534,21:00,Eua Ncaa,Manhattan,Niagara,2.28,Back Home
10548,14:00,Sérvia Primeira Liga,Tamis,Radnicki,1.61,Back Home
10558,09:00,Filipinas Copa Commissioners,Rain Or Shine Paint.,TNT Tropang Giga,1.75,Back Home
10560,05:30,Austrália Liga Feminina Wnbl,Adelaide Lightning F,Sydney F,2.08,Back Home
10604,00:00,Eua Ncaa,UC Riverside,UC Davis,1.73,Back Home
